In [23]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType

spark=SparkSession.builder.appName('FordGoBike').getOrCreate()
spark
# https://www.kaggle.com/code/priyankabnl/fordgobike-trip-data

In [24]:
path="H:/Drive'ım/Develhope/Develhope_stage2/develhope-Data5-Team3/Week_1/2017-fordgobike-tripdataa.csv"
df_pyspark=spark.read.csv(path,header=True, inferSchema=True)

In [7]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [8]:
df_pyspark.head(5)


[Row(start_time='57:39.7', start time hour=4, start time minute=57, start time seconds=40, end_time='12:50.2', end_time hour=3, end_time minute=12, end_time seconds=50, start_station_id=74, start_station_name='Laguna St at Hayes St', start_station_latitude=37.77643482, start_station_longitude=-122.426244, end_station_id=43, end_station_name='San Francisco Public Library (Grove St at Hyde St)', end_station_latitude=37.7787677, end_station_longitude=-122.4159292, bike_id=96, user_type='Customer', member_birth_year=1987, member_gender='Male', pyment='credit card'),
 Row(start_time='56:34.8', start time hour=3, start time minute=56, start time seconds=35, end_time='49:55.6', end_time hour=1, end_time minute=49, end_time seconds=56, start_station_id=284, start_station_name='Yerba Buena Center for the Arts (Howard St at 3rd St)', start_station_latitude=37.78487208, start_station_longitude=-122.4008757, end_station_id=96, end_station_name='Dolores St at 15th St', end_station_latitude=37.76621

In [25]:
df_pyspark=df_pyspark.withColumn('start_station_longitude',F.col('start_station_longitude').cast(FloatType()))\
    .withColumn('start_station_latitude',F.col('start_station_latitude').cast(FloatType()))\
    .withColumn('end_station_latitude',F.col('end_station_latitude').cast(FloatType()))\
    .withColumn('end_station_longitude',F.col('end_station_longitude').cast(FloatType()))
df_pyspark.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- start time hour: integer (nullable = true)
 |-- start time minute: integer (nullable = true)
 |-- start time seconds: integer (nullable = true)
 |-- end_time: string (nullable = true)
 |-- end_time hour: integer (nullable = true)
 |-- end_time minute: integer (nullable = true)
 |-- end_time seconds: integer (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: float (nullable = true)
 |-- start_station_longitude: float (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: float (nullable = true)
 |-- end_station_longitude: float (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



In [26]:
df_pyspark.describe().show()


+-------+----------+------------------+------------------+------------------+--------+------------------+------------------+------------------+-----------------+--------------------+----------------------+-----------------------+-----------------+--------------------+--------------------+---------------------+------------------+----------+------------------+-------------+-----------+
|summary|start_time|   start time hour| start time minute|start time seconds|end_time|     end_time hour|   end_time minute|  end_time seconds| start_station_id|  start_station_name|start_station_latitude|start_station_longitude|   end_station_id|    end_station_name|end_station_latitude|end_station_longitude|           bike_id| user_type| member_birth_year|member_gender|     pyment|
+-------+----------+------------------+------------------+------------------+--------+------------------+------------------+------------------+-----------------+--------------------+----------------------+---------------------

In [ ]:
df_pyspark.na.drop(how='any', subset=['start_time']).show()


In [ ]:
df_pyspark.groupBy('start_station_id').count().show()

In [27]:
df_pyspark=df_pyspark.withColumn('start time',F.to_timestamp(F.concat_ws(':','start time hour','start time minute','start time seconds')))
df_pyspark=df_pyspark.withColumn('end time',F.to_timestamp(F.concat_ws(':','end_time hour','end_time minute','end_time seconds')))

In [28]:
df_pyspark=df_pyspark.withColumn('end time',F.when(df_pyspark['end time']<df_pyspark['start time'], df_pyspark['start time']).otherwise(df_pyspark['end time']))
df_pyspark=df_pyspark.withColumn('start time',F.when(df_pyspark['end time']==df_pyspark['start time'], F.to_timestamp(F.concat_ws(':','end_time hour','end_time minute','end_time seconds'))).otherwise(df_pyspark['start time']))

In [29]:
df_pyspark=df_pyspark.withColumn('start_time',F.concat_ws(':',F.hour('start time'),F.minute('start time'),F.second('start time')))\
    .withColumn('end_time',F.concat_ws(':',F.hour('end time'),F.minute('end time'),F.second('end time')))\
    .withColumn('start time hour',F.hour('start time')).withColumn('start time minute',F.minute('start time')).withColumn('start time seconds',F.second('start time'))\
    .withColumn('end_time hour',F.hour('end time')).withColumn('end_time minute',F.minute('end time')).withColumn('end_time seconds',F.second('start time'))

In [30]:
df_pyspark=df_pyspark.drop(*['start time','end time'])
df_pyspark.show()

+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|start_time|start time hour|start time minute|start time seconds|end_time|end_time hour|end_time minute|end_time seconds|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|
+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------

In [35]:
from haversine import haversine

def haversine_f(lat1, lon1, lat2, lon2):
    return haversine( (lat1, lon1), (lat2, lon2),unit='m',normalize=True )

haversine_udf = F.udf(haversine_f)

In [36]:
df_pyspark=df_pyspark.withColumn('haversine_distance', 
                    haversine_udf(F.col('start_station_latitude'), F.col('start_station_longitude'), 
                                  F.col('end_station_latitude'), F.col('end_station_longitude'))
                    )

In [37]:
df_pyspark.printSchema()

root
 |-- start_time: string (nullable = false)
 |-- start time hour: integer (nullable = true)
 |-- start time minute: integer (nullable = true)
 |-- start time seconds: integer (nullable = true)
 |-- end_time: string (nullable = false)
 |-- end_time hour: integer (nullable = true)
 |-- end_time minute: integer (nullable = true)
 |-- end_time seconds: integer (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: float (nullable = true)
 |-- start_station_longitude: float (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: float (nullable = true)
 |-- end_station_longitude: float (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 

In [41]:
df_pyspark=df_pyspark.withColumn('start_time',F.to_timestamp('start_time','HH:MM:SS'))\
    .withColumn('end_time',F.to_timestamp('end_time','HH:MM:SS'))\
    .withColumn('Diff_in_seconds',F.col('end_time').cast('long')-F.col('start_time').cast('long'))\
    .withColumn('Diff_in_minutes',(F.col('Diff_in_seconds')/60))\
    .withColumn('Trip_cost',(F.col('Diff_in_minutes')*0.35))

df_pyspark.printSchema()


root
 |-- start_time: timestamp (nullable = true)
 |-- start time hour: integer (nullable = true)
 |-- start time minute: integer (nullable = true)
 |-- start time seconds: integer (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- end_time hour: integer (nullable = true)
 |-- end_time minute: integer (nullable = true)
 |-- end_time seconds: integer (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: float (nullable = true)
 |-- start_station_longitude: float (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: float (nullable = true)
 |-- end_station_longitude: float (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = tru